# Generating a Grafana Dashboard with grafwiz

This tutorial demonstrates how to use [grafwiz](https://github.com/v3io/grafwiz), Iguazio's open-source Python library for generating a Grafana dashboard programmatically.

- [Setup](#grafwiz-setup)
- [Generating Data](#grafwiz-gen-data)
- [Creating a DataFrame with the Generated Data](#grafwiz-df-create)
- [Writing the Data to the Platform's Data Store](#grafwiz-write-to-data-store)
- [Adding a Platform Data Source to Grafana](#grafwiz-add-data-source)
- [Creating a Grafana Dashboard](#grafwiz-grafana-dashboard-create)
- [Adding Dashboard Visualization Elements](#grafwiz-add-dashboard-visualization-elements)
- [Deploying the Dashboard to Grafana](#grafwiz-grafana-dashboard-deploy)

<a id="grafwiz-setup"></a>

## Setup

Initialize and configure your environment.

<a id="grafwiz-install"></a>

### Installing grafwiz

Ensure that the `grafwiz` Pyhton package is installed.

In [ ]:
!pip install git+https://github.com/v3io/grafwiz --upgrade

<a id="grafwiz-setup-grafana"></a>

### Creating a Grafana Service

1.  Ensure that you have a running platform Grafana service.
    You can create such a service from the platform dashboard's **Services** page.
2.  Copy the URL of your Grafana service from the **API** column in the services table on the **Services** dashboard page.

<a id="grafwiz-define-variables"></a>

### Defining Variables

Define variables for your environment.

In [ ]:
import os

grafana_url = 'http://grafana' 
v3io_container = 'users'
stocks_kv_table = os.path.join(os.getenv("V3IO_USERNAME"),'stocks_kv_table')
stocks_tsdb_table = os.path.join(os.getenv("V3IO_USERNAME"),'stocks_tsdb_table')
sym = 'XYZ'
rows = 3450

<a id="grafwiz-import-libs"></a>

### Importing Libraries

Import required libraries.

In [ ]:
from grafwiz import *
import v3io_frames as v3f
import pandas as pd

<a id="grafwiz-create-frames-client"></a>

### Creating a V3IO Frames Client

Create a V3IO Frames client object.

In [ ]:
client = v3f.Client('framesd:8081',container=v3io_container)

<a id="grafwiz-gen-data"></a>

## Generating Data

Generate random data to visualize on the Grafana dashboard.

In [ ]:
import random
import datetime
import numpy as np

def generate_date(rows):
    
    datetimes = [datetime.datetime.today() - (random.random() * datetime.timedelta(minutes=15)) for i in range(rows)]
    return datetimes

time = sorted(generate_date(rows))
volume = np.random.randint(low=100, high=10000, size=rows)
price = np.cumsum([0.0001] * rows + np.random.random(rows))

<a id="grafwiz-df-create"></a>

## Creating a DataFrame with the Generated Data

Store the generated data in a pandas DataFrame.

In [ ]:
stocks_df = pd.DataFrame(
    {'last_updated': time,
     'volume': volume,
     'price': price
    })
stocks_df['symbol'] = sym
stocks_df = stocks_df.sort_values('last_updated')
stocks_df

Define the `last_updated` column (attribute) as a DataFrame index column, which will be used to identify the ingestion times of the TSDB metric samples.

In [ ]:
stocks_df_tsdb = stocks_df
stocks_df_tsdb = stocks_df.reset_index()
stocks_df_tsdb = stocks_df.set_index(['last_updated'])

<a id="grafwiz-write-to-data-store"></a>

## Writing the Data to the Platform's Data Store

Use the V3IO Frames API to write the data from the pandas DataFrame to TSDB and NoSQL tables in the platform's persistent data store.

<a id="grafwiz-tsdb-table-write"></a>

### Writing the Data to a TSDB Table

Write the data from the DataFrame to a new platform TSDB table.

In [ ]:
client.create(backend='tsdb', table=stocks_tsdb_table, rate='1/m', if_exists=1)
client.write(backend='tsdb', table=stocks_tsdb_table, dfs=stocks_df_tsdb)

<a id="grafwiz-nosql-table-write"></a>

### Writing the Data to a NoSQL Table

Write the data from the DataFrame to a new platform NoSQL table in order of rows arrival, to simulate real-time data consumption.

In [ ]:
expr_template = "symbol='{symbol}';price='{price}';volume='{volume}';last_updated='{last_updated}'"
# Write the stock data to a NoSQL table
for idx, record in stocks_df.iterrows():
    stock = {'symbol': sym, 'price': record['price'], 'volume': record['volume'], 'last_updated': record['last_updated']}
    expr = expr_template.format(**stock)
    client.execute('kv', stocks_kv_table, 'update', args={'key': sym, 'expression': expr})

Infer the schema of the NoSQL table to verify that it can be accessed and displayed on the dashboard.

In [ ]:
# Infer the schema of the NoSQL table
client.execute(backend='kv', table=stocks_kv_table, command='infer')

<a id="grafwiz-add-data-source"></a>

## Adding a Platform Data Source to Grafana

Add an "Iguazio" data source for the platform's custom `iguazio` Grafana data source to your Grafana service.

In [ ]:
# Create a data source
DataSource(name='Iguazio').deploy(grafana_url, use_auth=True)

<a id="grafwiz-grafana-dashboard-create"></a>

## Creating a Grafana Dashboard

Create a new Grafana dashboard that uses the platform's `iguazio` data source.

In [ ]:
# Create grafana dashboard
dash = Dashboard("stocks", start='now-15m', dataSource='Iguazio', end='now')

<a id="grafwiz-add-dashboard-visualization-elements"></a>

## Adding Dashboard Visualization Elements

Create a table and two graphs for visualization the data from the NoSQL and TSDB tables on the Grafana dashboard.

In [ ]:
# Create a table and log viewer for the NoSQL table in one row
tbl = Table('Current Stocks Value', span=12).source(table=stocks_kv_table,fields=['symbol','volume', 'price', 'last_updated'],container=v3io_container)
dash.row([tbl])

# Create a TSDB metrics graph
metrics_row = [Graph(metric).series(table=stocks_tsdb_table, fields=[metric], container=v3io_container) for metric in ['price','volume']]
dashboard = dash.row(metrics_row)

<a id="grafwiz-grafana-dashboard-deploy"></a>

## Deploying the Dashboard to Grafana

Deploy the new Grafana dashboard to your Grafana service.

> **Note:** Allow a few minutes for the dashboard loading to complete. 

In [ ]:
# Deploy to Grafana
dash.deploy(grafana_url)